In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN,Dense,Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

,Earth is the third planet from the Sun and the only astronomical object known to harbor life. This is enabled by Earth being a water world,the only one in the Solar System sustaining liquid surface water. Almost all of Earth's water is contained in its global ocean,covering 70.8% of Earth's crust. The remaining 29.2% of Earth's crust is land,most of which is located in the form of continental landmasses within one hemisphere,Earth's land hemisphere. Most of Earth's land is somewhat humid and covered by vegetation,while large sheets of ice at Earth's polar deserts retain more water than Earth's groundwater,lakes,rivers and atmospheric water combined. Earth's crust consists of slowly moving tectonic plates,which interact to produce mountain ranges,vol…


In [ ]:
#generating some ex sequential data
sentences=['I love programming','I am vaishnavi','i have some data','Ihave idea about phyton','im a student']

In [ ]:


#Tokenizing the words
tokenizer= Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words=len(tokenizer.word_index)
print(total_words)

15


In [ ]:
# Creating input sequences and their corresponding next words
input_sequences = []
for sentence in sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        n_gram_sequence = tokenized_sentence[:i+1]
        input_sequences.append(n_gram_sequence)
input_sequences

[[1, 2],
 [1, 2, 3],
 [1, 4],
 [1, 4, 5],
 [1, 6],
 [1, 6, 7],
 [1, 6, 7, 8],
 [9, 10],
 [9, 10, 11],
 [9, 10, 11, 12],
 [13, 14],
 [13, 14, 15]]

In [ ]:
# Padding sequences for consistent input size
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
input_sequences

array([[ 0,  0,  1,  2],
       [ 0,  1,  2,  3],
       [ 0,  0,  1,  4],
       [ 0,  1,  4,  5],
       [ 0,  0,  1,  6],
       [ 0,  1,  6,  7],
       [ 1,  6,  7,  8],
       [ 0,  0,  9, 10],
       [ 0,  9, 10, 11],
       [ 9, 10, 11, 12],
       [ 0,  0, 13, 14],
       [ 0, 13, 14, 15]], dtype=int32)

In [ ]:
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words+1)

In [ ]:
# Building a simple RNN model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=50, input_length=max_sequence_length-1))
model.add(SimpleRNN(100, return_sequences=True))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
#model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#compiling the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#training Model
model.fit(X,y,epochs=50,verbose=2)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:

# Generating text using the trained model
seed_text = input("Enter the starting word: ")
next_words = int(input("Enter how many words to predict: "))

for _ in range(next_words):
    tokenized_seed = tokenizer.texts_to_sequences([seed_text])[0]
    tokenized_seed = pad_sequences([tokenized_seed], maxlen=max_sequence_length-1, padding='pre')
    predicted_word_index = np.argmax(model.predict(tokenized_seed), axis=-1)
    predicted_word = tokenizer.index_word[predicted_word_index[0]]
    seed_text += " " + predicted_word

print(seed_text)